### DLT Works with Three types of Datasets.
#### Streaming Tables (Temporary/Permananet) - Used as Append Data Sources, Incremental Data.
#### Materialized Views - Used for Transformations, Aggregations or Computations.
#### Views - Used for Intermediate Transformations, not stored in Target Schema.

- DLT pipelines are powered by Delta Lake
- We can't use All Purpose Compute to trigger our DLT pipelines, requires Job Compute.

In [0]:
import dlt

In [0]:
# Create a streaming table for Orders

@dlt.table(
    table_properties = {"quality": "bronze"},
    comment = "Orders Bronze Table"
)
def orders_bronze(): # by default the function name is the table name.
    # need a streaming source to create a streaming table.
    # delta tables have a property to work as input source for both streaming and batch modes.
    df = spark.readStream.table("syk_poc.`test-pranav`.orders_raw")

    return df 

In [0]:
# Create a Materialized View for Customers

@dlt.table(
    table_properties = {"quality": "bronze"},
    comment = "Customer Bronze Table",
    name = "customer_bronze"
)
def cust():
    # for a Materialized View the input source is a batch, not a stream.
    df = spark.read.table("syk_poc.`test-pranav`.customer_raw")

    return df 

In [0]:
# Create a View to join orders with Customers

# DLT Views are temporary and are used for intermediate transformation, they are not registered as targets.
# We use the LIVE Keyword to reference the DLT tables and views created in the same pipeline.

# @dlt.table -> Streaming table and Materialized Views
# @dlt.view -> for DLT Views

@dlt.view(
    comment = "Joined View",
)
def joined_vw():
    # for a Materialized View the input source is a batch, not a stream.
    df_c = spark.read.table("LIVE.customer_bronze")
    df_o = spark.read.table("LIVE.orders_bronze")

    df_join = df_o.join(df_c, how = "left_outer", on = df_c.c_custkey == df_o.o_custkey)

    return df_join

In [0]:
# Create a Materialized View to add a new column

from pyspark.sql.functions import current_timestamp

@dlt.table(
    table_properties = {"quality": "silver"},
    comment = "Joined Table",
    name = "joined_silver"
)
def joined_silver():
    # for a Materialized View the input source is a batch, not a stream.
    df = spark.read.table("LIVE.joined_vw").withColumn("__insert_date", current_timestamp())

    return df 

In [0]:
# Aggregate based on c_mktsegment and find the count of order (o_orderkey)

from pyspark.sql.functions import count

@dlt.table(
    table_properties = {"quality": "gold"},
    comment = "Aggregated Orders Table"
)
def orders_agg_gold():
    df = spark.read.table("LIVE.joined_silver")
    
    df_final = df.groupBy("c_mktsegment").agg(count("o_orderkey").alias("sum_orders")).withColumn("__insert_date", current_timestamp())

    return df_final 